## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-28-18-58-36 +0000,nyt,Abrupt Change for Warner Bros. Prompts Many Gr...,https://www.nytimes.com/2026/02/28/business/wa...
1,2026-02-28-18-57-29 +0000,nyt,Live Updates: U.S. and Israel Strike Iran as T...,https://www.nytimes.com/live/2026/02/28/world/...
2,2026-02-28-18-52-41 +0000,nyt,Iran Says Dozens Are Killed in Strike on School,https://www.nytimes.com/2026/02/28/world/middl...
3,2026-02-28-18-52-28 +0000,nypost,Kim Jong Un gifts sniper rifles to officials —...,https://nypost.com/2026/02/28/world-news/north...
4,2026-02-28-18-51-53 +0000,nypost,NYC Mayor Zohran Mamdani condemns Iran strike:...,https://nypost.com/2026/02/28/us-news/nyc-mayo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2588/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,iran,61
12,trump,47
40,strikes,25
73,attack,16
9,israel,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
129,2026-02-28-10-14-09 +0000,nypost,"Read, watch President Trump’s full speech afte...",https://nypost.com/2026/02/28/us-news/read-pre...,184
147,2026-02-28-07-10-48 +0000,nypost,"US, Israel launch ‘Operation Epic Fury’ attack...",https://nypost.com/2026/02/28/us-news/us-israe...,182
17,2026-02-28-18-23-54 +0000,wapo,"See where U.S., Israeli strikes have hit Iran ...",https://www.washingtonpost.com/investigations/...,169
9,2026-02-28-18-46-16 +0000,wapo,"Live updates: U.S., Israel strike Iran as Trum...",https://www.washingtonpost.com/world/2026/02/2...,169
148,2026-02-28-07-02-26 +0000,latimes,U.S. and Israel attack Iran; Trump calls for r...,https://www.latimes.com/world-nation/story/202...,164


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
129,184,2026-02-28-10-14-09 +0000,nypost,"Read, watch President Trump’s full speech afte...",https://nypost.com/2026/02/28/us-news/read-pre...
212,93,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...
242,69,2026-02-27-21-50-27 +0000,nypost,Warner Bros. Discovery CEO David Zaslav calls ...,https://nypost.com/2026/02/27/business/warner-...
11,59,2026-02-28-18-40-55 +0000,nypost,Nancy Mace bashes Ilhan Omar over Iran attack ...,https://nypost.com/2026/02/28/us-news/nancy-ma...
278,46,2026-02-27-19-54-00 +0000,wsj,The Pentagon said it shot down a “seemingly th...,https://www.wsj.com/politics/national-security...
155,42,2026-02-28-04-07-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...
147,42,2026-02-28-07-10-48 +0000,nypost,"US, Israel launch ‘Operation Epic Fury’ attack...",https://nypost.com/2026/02/28/us-news/us-israe...
208,42,2026-02-27-23-24-00 +0000,wsj,Kristi Noem for months failed to respond to an...,https://www.wsj.com/politics/national-security...
51,40,2026-02-28-17-03-00 +0000,nypost,Punch the monkey and new pal cozy up to stuffe...,https://nypost.com/2026/02/28/world-news/punch...
77,39,2026-02-28-15-04-01 +0000,nypost,Saudi Arabia condemns Iran’s attacks on Gulf n...,https://nypost.com/2026/02/28/us-news/saudi-ar...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
